## Import Libraries

In [9]:
from pyspark.sql.functions import hour,minute,second,col,avg,when
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlFun
import datetime
from filterAllData import *
sqlContext = SQLContext(sc)

## Read csv file from hdfs server

In [12]:
df_log = sqlContext.read.csv("hdfs:///user/spark-file/output",header = True, inferSchema = True)
df_log.show(10)

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|         start_time|           end_time|          idle_time|       working_hour|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|ashutoshrit64@gma...|2019-10-21 08:50:01|2019-10-21 19:25:02|2019-10-21 00:30:00|2019-10-21 10:05:01|
|giridhardandikwar...|2019-10-21 15:50:01|2019-10-21 18:05:02|2019-10-21 00:00:00|2019-10-21 02:15:01|
|priyankagorte95@g...|2019-10-21 10:40:01|2019-10-21 19:25:01|2019-10-21 00:45:00|2019-10-21 08:00:00|
|salinabodale73@gm...|2019-10-21 11:25:01|2019-10-21 18:25:02|2019-10-21 01:20:00|2019-10-21 05:40:01|
|“shivnajalisangal...|2019-10-21 10:25:01|2019-10-21 19:25:01|2019-10-21 08:55:00|2019-10-21 00:05:00|
|Krushnanikam26@gm...|2019-10-21 11:15:02|2019-10-21 13:45:01|2019-10-21 01:35:00|2019-10-21 00:54:59|
|gaikwadr576@gmail...|2019-10-21 09:55:01|2019-10-21 19:15:01|2019-10-21 

In [13]:
print('Rows : ',df_log.count(), 'Columns : ',len(df_log.columns))    # total no of row and column

Rows :  307 Columns :  5


### Find total number of hours for each user in working hours 

In [15]:
df_work = df_log.drop('idle_time', 'start_time', 'end_time')

In [16]:
df_avg = df_work.groupBy('user_name').agg(sqlFun.from_unixtime(sqlFun.avg(sqlFun.unix_timestamp('working_hour')),'hh:mm:ss').alias('avg_time'))
df_avg.show(5,truncate=50)

+---------------------------+--------+
|                  user_name|avg_time|
+---------------------------+--------+
|    ashutoshrit64@gmail.com|08:56:15|
|giridhardandikwar@gmail.com|12:30:00|
|  priyankagorte95@gmail.com|05:10:00|
|              nikitapawar17|02:20:00|
|   salinabodale73@gmail.com|04:56:15|
+---------------------------+--------+
only showing top 5 rows



In [17]:
df_avg_hours = df_avg.withColumn('avg_hour', (hour(df_avg['avg_time'])*3600 + minute(df_avg['avg_time'])*60 + second(df_avg['avg_time']))/3600)
df_avg_hours.show(5,truncate=50)

+---------------------------+--------+------------------+
|                  user_name|avg_time|          avg_hour|
+---------------------------+--------+------------------+
|    ashutoshrit64@gmail.com|08:56:15|            8.9375|
|giridhardandikwar@gmail.com|12:30:00|              12.5|
|  priyankagorte95@gmail.com|05:10:00| 5.166666666666667|
|              nikitapawar17|02:20:00|2.3333333333333335|
|   salinabodale73@gmail.com|04:56:15|            4.9375|
+---------------------------+--------+------------------+
only showing top 5 rows



In [19]:
#calculating average hours
total_avg_work_hour = df_avg_hours.select(avg('avg_hour')).collect()[0][0]
total_avg_work_hour

6.318108187134503

In [20]:
avg_count = df_avg_hours.filter(sqlFun.col("avg_hour") > total_avg_work_hour)
avg_count.count()

48

In [21]:
df_avg_hours.filter(df_avg_hours['avg_hour'] > total_avg_work_hour).select('user_name').show(5,truncate=50)

+--------------------------------+
|                       user_name|
+--------------------------------+
|         ashutoshrit64@gmail.com|
|     giridhardandikwar@gmail.com|
|“shivnajalisangale5456@gmail.com|
|     indrajeetgajbhiye@gmail.com|
|      khairnarswapna99@gmail.com|
+--------------------------------+
only showing top 5 rows

